## Data Processing Stage-2 [Preparing Tefla ready data]

This Notebook runs after the DataProcessing-1.ipynb notebook. This notebook creates the csv file with consists of image name and its corresponding label. Then it prepares the data in the tefla ready format, which the adequate distribution of data between training, validation, and test for all the categories.

In [1]:
import os
import pandas as pd
import math
import numpy as np
from PIL import Image
import glob

In [2]:
classes = ["Alzheimer", "MCI", "Normal"]
#sourceDir = '/home/nagdev/work/siddhant/data/raw/'
sourceDir = '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/'
path_tocsvfile = '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/'

# 0 represents Alzheimer class, 1 represents MCI class, 2 represents Normal class.
count = 0
with open(path_tocsvfile + 'all.csv','a') as o:
    o.write('nii' + ',' + 'label' +'\n')
    for class_ in classes:
        images_path = os.path.join(os.path.join(sourceDir,class_), '*.nii')
        print 'Working on: ', images_path
        for file_ in glob.glob(images_path):
            o.write(file_ + ',' + str(count) +'\n')
        count += 1
        print count

Working on:  /home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/Alzheimer/*.nii
1
Working on:  /home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/MCI/*.nii
2
Working on:  /home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/Normal/*.nii
3


In [4]:
import os
import pandas as pd
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# script to create a tefla compatible data dir for training and validation data
sourceDir = '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/'
destDir = '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/'

categories_picked_for_validation_data = [0,1,2]
categories_picked_for_test_data = [0,1,2]
validation_data_percentage = 10
test_data_percentage = 20

def create_tefla_data(source_dir,destination_dir,validation_categories,test_categories,validation_percentage,test_percentage):
    #if os.path.exists(destination_dir):
    #    shutil.rmtree(destination_dir)

    training_dir = destination_dir + 'training_224/'
    validation_dir = destination_dir + 'validation_224/'
    test_dir = destination_dir + 'test_224/'
    
    try:
        os.makedirs(training_dir)
    except:
        pass
    try:
        os.makedirs(validation_dir)
    except:
        pass
    try:
        os.makedirs(test_dir)
    except:
        pass
    
    labels = pd.read_csv( source_dir + 'all.csv')
    print labels.head()
    validation_dict = {}
    test_dict = {}

    for c in validation_categories:
        validation_dict[c] = []

    for c in test_categories:
        test_dict[c] = []


    # test set creation
    test_set = []

    for i, rows in labels.iterrows():
        if test_dict.has_key(rows['label']):
            test_dict[rows['label']].append(rows['nii'])
    print test_dict
    for k in test_dict:
        np.random.seed(0)
        n = math.ceil(test_percentage * len(test_dict[k]) / 100.0)
        random_array = np.random.choice(test_dict[k],int(n))
        test_set = test_set + random_array.tolist()
    
    print test_set
    # now test_set has some nii files inside it from each class

    #validation set creation
    validation_set = []

    for i, rows in labels.iterrows():
        if rows['nii'] not in test_set:
            if validation_dict.has_key(rows['label']):
                validation_dict[rows['label']].append(rows['nii'])

    for l in validation_dict:
        np.random.seed(0)
        n = math.ceil(validation_percentage * len(validation_dict[l]) / 100.0)
        random_array = np.random.choice(validation_dict[l],int(n))
        validation_set = validation_set + random_array.tolist()
    print validation_set
    # add some of the nii files to validation set which are not part of test set.

    training_images = []
    training_labels = []
    validation_images = []
    validation_labels =[]
    test_images = []
    test_labels = []

    for i, rows in labels.iterrows():
        if rows['nii'] in test_set:
            #print rows['nii']
            for file_ in glob.glob(os.path.join(rows['nii'], '*.jpg')):
                test_images.append(os.path.basename(file_).replace('.jpg', ''))
                test_labels.append(rows['label'])
                #print file_
                process_and_save_image(file_, test_dir + os.path.basename(file_).replace('.jpg','') + ".tiff")
        elif rows['nii'] in validation_set:
            #print rows['nii']
            for file_ in glob.glob(os.path.join(rows['nii'], '*.jpg')):
                validation_images.append(os.path.basename(file_).replace('.jpg', ''))
                validation_labels.append(rows['label'])
                #print file_
                process_and_save_image(file_, validation_dir + os.path.basename(file_).replace('.jpg','') + ".tiff")
        else:
            #print rows['nii']
            for file_ in glob.glob(os.path.join(rows['nii'], '*.jpg')):
                training_images.append(os.path.basename(file_).replace('.jpg', ''))
                training_labels.append(rows['label'])
                process_and_save_image(file_, training_dir + os.path.basename(file_).replace('.jpg','') + ".tiff")
                
    header = ['image', 'label']

    # saving training csv
    training_out = np.column_stack((training_images, training_labels))
    training_out = np.row_stack((header, training_out))
    np.savetxt(destination_dir + 'training_labels.csv', training_out, delimiter=',', fmt='%s')

    # saving validation csv
    validation_out = np.column_stack((validation_images, validation_labels))
    validation_out = np.row_stack((header, validation_out))
    np.savetxt(destination_dir + 'validation_labels.csv', validation_out, delimiter=',', fmt='%s')

    # saving testing csv
    test_out = np.column_stack((test_images, test_labels))
    test_out = np.row_stack((header, test_out))
    np.savetxt(destination_dir + 'test_labels.csv', test_out, delimiter=',', fmt='%s')

def process_and_save_image(source_path,destination_path):
    img = resize(source_path, 224)
    img.save(destination_path, quality=100)

def resize(fname, target_size):
    # print('Processing image: %s' % fname)
    img = Image.open(fname)
    return img

#calling method
create_tefla_data(sourceDir,destDir,categories_picked_for_validation_data,categories_picked_for_test_data,validation_data_percentage,test_data_percentage)



                                                 nii  label
0  /home/h_hack/work/pywork/ML/datasets/Alzheimer...      0
1  /home/h_hack/work/pywork/ML/datasets/Alzheimer...      0
2  /home/h_hack/work/pywork/ML/datasets/Alzheimer...      0
3  /home/h_hack/work/pywork/ML/datasets/Alzheimer...      0
4  /home/h_hack/work/pywork/ML/datasets/Alzheimer...      0
{0: ['/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/Alzheimer/A26_018_S_4733_result.nii', '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/Alzheimer/A6_130_S_4997_result.nii', '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/Alzheimer/A28_013_S_5071_result.nii', '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/Alzheimer/A2_131_S_5138_result.nii', '/home/h_hack/work/pywork/ML/datasets/Alzheimer_detect/fsl_preprocessed/images/Alzheimer/A8_130_S_4984_result.nii'], 1: ['/home/h_hack/work/pywork/ML/datasets/Alzheimer_